# MATPOWER IEEE Case 9

R.P. Schulz, A.E. Turner and D.N. Ewart, "Long Term Power System Dynamics," EPRI Report 90-7-0, Palo Alto, California, 1974.

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## DC power flow with fixed power

In [8]:
import os
import numpy as np

from GridCal.Engine import Bus, Load, Line, Generator, MultiCircuit, StaticGenerator
from GridCal.Engine import PowerFlowOptions, PowerFlowDriver, SolverType
from GridCal.Engine.IO import parse_matpower_file

from matpower import path_matpower

import pandas as pd
pd.options.display.float_format = '{:.3f}'.format

In [9]:
def round_power_close_to_zero(power):
    real_power = np.real(power)
    real_power[np.isclose(real_power,0,atol=1e-06)] = 0

    imag_power = np.imag(power)
    imag_power[np.isclose(imag_power,0,atol=1e-06)] = 0

    return real_power, imag_power

In [10]:
case_name = 'case9.m'
case_path = os.path.join(path_matpower, 'data', case_name)

grid, _ = parse_matpower_file(case_path)

In [11]:
for bus in grid.buses:
    bus.is_dc = True
    for load in bus.loads:
        load.Q = 0
    if bus.is_slack:
        continue
    for controlled_generator in bus.controlled_generators:
        static_generator = StaticGenerator(
            P=controlled_generator.P,
            Q=0,
        )
        bus.static_generators.append(static_generator)
    bus.controlled_generators = []

for line in grid.lines:
    line.X = 0
    line.B = 0
    if line.R == 0:
        line.R = 0.001

In [12]:
options = PowerFlowOptions(SolverType.NR, verbose=False)
power_flow = PowerFlowDriver(grid, options)
power_flow.run()

In [13]:
print('|V|:', abs(power_flow.results.voltage))
print('|Sbranch|:', abs(power_flow.results.Sf))
print('|loading|:', abs(power_flow.results.loading) * 100)
print('err:', power_flow.results.error)
print('Conv:', power_flow.results.converged)
print('Total real losses:', sum(np.real(power_flow.results.losses)))
print('Power flow:', power_flow.results.If)

|V|: [1.04       1.06250648 1.05814743 1.0393119  1.03448583 1.05734414 1.0547597  1.06097237 1.03526981]
|Sbranch|: [ 71.56202309  29.5046965   60.63230947  85.00000012  22.96341314  77.09271596 162.76465148  85.21784905  41.8465943 ]
|loading|: [28.62480924 11.8018786  40.42153964 28.33333337 15.3089421  30.83708638 65.10586059 34.08713962 16.73863772]
err: 7.255492477491056e-09
Conv: True
Total real losses: 4.5620240742523555
Power flow: [ 0.68809638+0.j  0.28388683+0.j -0.58611059+0.j  0.80329071+0.j  0.21718012+0.j -0.73090313+0.j -1.53410829+0.j  0.80320517+0.j -0.40420955+0.j]


In [14]:
df_bus, df_branch = power_flow.results.export_all()
df_branch['Imag power (MVAr)'] = abs(df_branch['Imag power (MVAr)'])
df_branch

,Real power (MW),Imag power (MVAr),Power module (MVA),Loading(%),Losses (MW),Losses (MVAr),Losses (MVA),Tap module
0,71.562,0.000,71.562,0.286,0.047,0.000,0.047,NaN
1,29.505,0.000,29.505,0.118,0.137,0.000,0.137,NaN
2,-60.632,0.000,60.632,0.404,1.340,0.000,1.340,NaN
3,85.000,0.000,85.000,0.283,0.065,0.000,0.065,NaN
4,22.963,0.000,22.963,0.153,0.056,0.000,0.056,NaN
5,-77.093,0.000,77.093,0.308,0.454,0.000,0.454,NaN
6,-162.765,0.000,162.765,0.651,0.235,0.000,0.235,NaN
7,85.218,0.000,85.218,0.341,2.064,0.000,2.064,NaN
8,-41.847,0.000,41.847,0.167,0.163,0.000,0.163,NaN


In [15]:
df_bus['Real power (MW)'], df_bus['Imag power (MVAr)'] = round_power_close_to_zero(power_flow.results.Sbus)
df_bus


,Real voltage (p.u.),Imag Voltage (p.u.),Voltage module (p.u.),Voltage angle (rad),Real power (MW),Imag power (MVAr)
0,1.040,0.000,1.040,0.000,71.562,0.000
1,1.063,0.000,1.063,0.000,163.000,0.000
2,1.058,0.000,1.058,0.000,85.000,0.000
3,1.039,0.000,1.039,0.000,0.000,0.000
4,1.034,0.000,1.034,0.000,-90.000,0.000
5,1.057,0.000,1.057,0.000,0.000,0.000
6,1.055,0.000,1.055,0.000,-100.000,0.000
7,1.061,0.000,1.061,0.000,0.000,0.000
8,1.035,0.000,1.035,0.000,-125.000,0.000
